## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from stargazer.stargazer import Stargazer
from pystout import pystout
from IPython.display import Latex
from IPython.core.display import display, HTML
from statsmodels.regression.linear_model import RegressionResultsWrapper
from statsmodels.iolib.summary2 import summary_col

# Modeling
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Tests
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score
from sklearn.feature_selection import VarianceThreshold, RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
class Logit_model:
    
    def __init__(self, X, y):
        self.X0, self.X1, self.Y0, self.Y1 = train_test_split(X, y, test_size=0.25, random_state=42)
        self.X = X
        self.y = y

    def skLogit(self):
        result = LogisticRegression(random_state=42, class_weight='balanced', 
                                    max_iter=200).fit(self.X0,self.Y0)
        return result
    
    def statLogit(self, maxiter=70):
        result = sm.Logit(self.Y0, self.X0).fit(maxiter=maxiter)
        return result

    def model_performance(self):
        model = self.skLogit()
        
        Y0_predict = model.predict(self.X0)
        Y1_predict = model.predict(self.X1)

        Y0_hat = model.predict_proba(self.X0)[:, 1]
        Y1_hat = model.predict_proba(self.X1)[:, 1]

        print('\nROC AUC Train', roc_auc_score(self.Y0, Y0_hat).round(3))
        print('ROC AUC Test', roc_auc_score(self.Y1, Y1_hat).round(3))
        print('-------------------------')
        print('\nClassification Report Train')
        print(classification_report(self.Y0, Y0_predict))
        print('-------------------------')
        print('\nClassification Report Test')
        print(classification_report(self.Y1, Y1_predict))
        print('-------------------------')

        cnf_matrix = confusion_matrix(self.Y1, Y1_predict)
        class_names = [0, 1]  # name  of classes
        fig, ax = plt.subplots()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names)
        plt.yticks(tick_marks, class_names)
        # create heatmap
        sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu", fmt='g')
        ax.xaxis.set_label_position("top")
        plt.tight_layout()
        plt.title('Confusion matrix', y=1.1)
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')

    def vif(self):
        # Calculating VIF
        vifc = pd.DataFrame()
        vifc["variables"] = self.X.columns
        vifc["VIF"] = [variance_inflation_factor(self.X.values, i) for i in range(self.X.shape[1])]
        vifc = vifc.sort_values(by='VIF', ascending=False).reindex()
        return(vifc)

    def best_features(self, n_features):
        estimator = self.skLogit()
        selector =  RFE(estimator, n_features, step=1)
        selector.fit(self.X0, self.Y0)
        n_feature_rankings = 1
        best_features = self.X0.columns[selector.ranking_ <= n_feature_rankings]
        return best_features

def low_freq_combine(data, columns, threshold_percent=1):
    data = data.copy()
    for column in columns:
        series = pd.value_counts(data[column])
        mask = (series/series.sum() * 100).lt(threshold_percent)
        low_freq_updated = np.where(df[column].isin(series[mask].index), 'Other', df[column])
        data[column] = low_freq_updated
    return data

def explain_components(price_dummies, dummy_comps_explain):
    main_comp_corrls = pd.concat([price_dummies, dummy_comps_explain], axis=1).corr()[dummy_comps_explain.columns]
    
    comps_explained = []
    for c in dummy_comps_explain.columns:
        compParts = main_comp_corrls.sort_values(by=c, ascending=False).index
        comps_explained.append(compParts)

    comps_explained_df = pd.DataFrame(comps_explained).transpose()
    comps_explained_df.columns = dummy_comps_explain.columns
    return comps_explained_df

## Data Cleanup

In [3]:
purchaseData = pd.read_csv('./data/purchase data.csv')
segmentationData = pd.read_csv('./data/segmentation-data.csv')

purchaseData['ID'] = purchaseData.ID - 100000000

data = purchaseData.merge(segmentationData)

In [4]:
data.head()

,ID,Day,Incidence,Brand,Quantity,Last_Inc_Brand,Last_Inc_Quantity,Price_1,Price_2,Price_3,...,Promotion_3,Promotion_4,Promotion_5,Sex,Marital status,Age,Education,Income,Occupation,Settlement size
0,100000001,1,0,0,0,0,0,1.59,1.87,2.01,...,0,0,0,0,0,67,2,124670,1,2
1,100000001,11,0,0,0,0,0,1.51,1.89,1.99,...,0,0,0,0,0,67,2,124670,1,2
2,100000001,12,0,0,0,0,0,1.51,1.89,1.99,...,0,0,0,0,0,67,2,124670,1,2
3,100000001,16,0,0,0,0,0,1.52,1.89,1.98,...,0,0,0,0,0,67,2,124670,1,2
4,100000001,18,0,0,0,0,0,1.52,1.89,1.99,...,0,0,0,0,0,67,2,124670,1,2


In [5]:
dummies = ['Incidence', 'Brand', 'Last_Inc_Brand', 'Last_Inc_Quantity', 
          'Promotion_1', 'Promotion_2', 'Promotion_3', 'Promotion_4', 
          'Promotion_5', 'Sex', 'Marital status', 'Education', 'Occupation', 
          'Settlement size']

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58693 entries, 0 to 58692
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 58693 non-null  int64  
 1   Day                58693 non-null  int64  
 2   Incidence          58693 non-null  int64  
 3   Brand              58693 non-null  int64  
 4   Quantity           58693 non-null  int64  
 5   Last_Inc_Brand     58693 non-null  int64  
 6   Last_Inc_Quantity  58693 non-null  int64  
 7   Price_1            58693 non-null  float64
 8   Price_2            58693 non-null  float64
 9   Price_3            58693 non-null  float64
 10  Price_4            58693 non-null  float64
 11  Price_5            58693 non-null  float64
 12  Promotion_1        58693 non-null  int64  
 13  Promotion_2        58693 non-null  int64  
 14  Promotion_3        58693 non-null  int64  
 15  Promotion_4        58693 non-null  int64  
 16  Promotion_5        586

In [7]:
data.isna().sum()

ID                   0
Day                  0
Incidence            0
Brand                0
Quantity             0
Last_Inc_Brand       0
Last_Inc_Quantity    0
Price_1              0
Price_2              0
Price_3              0
Price_4              0
Price_5              0
Promotion_1          0
Promotion_2          0
Promotion_3          0
Promotion_4          0
Promotion_5          0
Sex                  0
Marital status       0
Age                  0
Education            0
Income               0
Occupation           0
Settlement size      0
dtype: int64

In [8]:
data.to_csv('./data/charm_pricing_dataset.csv')

In [76]:
df = data.convert_dtypes()
df = pd.get_dummies(df, columns=dummies, drop_first=True)

In [77]:
# Getting Price dummies
prices = ['Price_1', 'Price_2', 'Price_3', 'Price_4', 'Price_5']

df_prices_updated = low_freq_combine(df, prices)
price_dummies = pd.get_dummies(df_prices_updated[prices])

# Adding price dummies to dataset
dataset = pd.concat([df, price_dummies], axis=1)

In [78]:
dataset['change_in_P1'] = dataset['Price_1'].pct_change().fillna(method='bfill')
dataset['change_in_P2'] = dataset['Price_2'].pct_change().fillna(method='bfill')
dataset['change_in_P3'] = dataset['Price_3'].pct_change().fillna(method='bfill')
dataset['change_in_P4'] = dataset['Price_4'].pct_change().fillna(method='bfill')
dataset['change_in_P5'] = dataset['Price_5'].pct_change().fillna(method='bfill')

In [79]:
# # Standardising non-dummies
# all_dummies = (dataset.dtypes[dataset.dtypes == 'uint8']).index.to_list()

# columns_NotD = dataset.drop(all_dummies, axis=1).columns

# scaler = StandardScaler()
# dataset[columns_NotD] = scaler.fit_transform(dataset[columns_NotD])

In [80]:
dataset.head()

,ID,Day,Quantity,Price_1,Price_2,Price_3,Price_4,Price_5,Age,Income,...,Price_5_2.77,Price_5_2.78,Price_5_2.79,Price_5_2.8,Price_5_Other,change_in_P1,change_in_P2,change_in_P3,change_in_P4,change_in_P5
0,100000001,1,0,1.59,1.87,2.01,2.09,2.66,67,124670,...,0,0,0,0,0,-0.05,0.01,-0.01,0.00,0.00
1,100000001,11,0,1.51,1.89,1.99,2.09,2.66,67,124670,...,0,0,0,0,0,-0.05,0.01,-0.01,0.00,0.00
2,100000001,12,0,1.51,1.89,1.99,2.09,2.66,67,124670,...,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00
3,100000001,16,0,1.52,1.89,1.98,2.09,2.66,67,124670,...,0,0,0,0,0,0.01,0.00,-0.01,0.00,0.00
4,100000001,18,0,1.52,1.89,1.99,2.09,2.66,67,124670,...,0,0,0,0,0,0.00,0.00,0.01,0.00,0.00


In [81]:
df_brand_3 = dataset[dataset[['Brand_1','Brand_2', 'Brand_4', 'Brand_5']].sum(axis=1).replace({0:1, 1:0}) == 1]

In [82]:
df_brand_3['Brand_3'].value_counts()

0    44055
1      841
Name: Brand_3, dtype: int64

## Modeling

### Model 1

In [83]:
# Using df_brand_3 and dataset
df_brand_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44896 entries, 0 to 58692
Columns: 131 entries, ID to change_in_P5
dtypes: Int64(5), float64(10), uint8(116)
memory usage: 10.7 MB


In [84]:
df_brand_3 = df_brand_3.astype('float')

target = 'Incidence_1'

y = df_brand_3[target]
X = df_brand_3.drop(target, axis=1)
X = sm.add_constant(X)

In [85]:
df_brand_3.corr()[target].sort_values(ascending=False)

Incidence_1          1.00
Brand_3              1.00
Quantity             0.86
Last_Inc_Brand_3     0.23
ID                   0.05
                     ... 
Settlement size_2   -0.03
Brand_1               nan
Brand_2               nan
Brand_4               nan
Brand_5               nan
Name: Incidence_1, Length: 131, dtype: float64

In [86]:
odd_price = []
even_price = []
price_ends_with_9 = []

for column in X[price_dummies.columns].columns:
    if column[:7] == 'Price_3':
        try:
            is_odd = int(column[-1]) % 2
            is_9 = column[-1] == '9'
            is_aeq_5 = int(column[-1]) >= 5
            if is_odd and is_aeq_5:
                odd_price.append(column)
            elif (not is_odd) and is_aeq_5:
                even_price.append(column)
            if is_9:
                price_ends_with_9.append(column)
        except:
            pass

X['ends_with_9'] = X[price_ends_with_9].sum(axis=1).apply(lambda x: int(x!=0)) 
X['odd_price_above_5'] = X[odd_price].sum(axis=1).apply(lambda x: int(x!=0))      
X['even_price_above_5'] = X[even_price].sum(axis=1).apply(lambda x: int(x!=0))

In [87]:
# Dropping columns that cause multicollinearity
excluding = ['ID', 'Brand_1', 'Brand_2', 'Brand_4', 'Brand_5', 'Brand_3', 
            'const', 'Quantity', 'Marital status_1', 'Education_1', 'Occupation_1',
             'Education_3', 'Price_5', 'Day', 'Price_4', 'Price_1', 'Price_2',
            'Income', 'Age', 'Promotion_4_1', 'Promotion_1_1', 'Last_Inc_Brand_5',
            'Promotion_5_1', 'Sex_1', 'Last_Inc_Brand_1', 'Last_Inc_Brand_4', 'Settlement size_1',
             'Promotion_2_1', 'Occupation_2',
            'ends_with_9', 'change_in_P4', 'change_in_P3']


other = ['Price_1_1.21', 'Price_1_1.33', 'Price_1_1.34', 'Price_1_1.35',
       'Price_1_1.42', 'Price_1_1.5', 'Price_1_Other', 'Price_2_1.9',
       'Price_5_2.64', 'Price_5_2.66', 'Price_5_2.7', 'Price_3_2.0', 
        'Price_3_1.95', 'Price_1_1.48', 'Price_3_1.91']

drop = ['Price_2_1.81', 'Price_4_1.9', 'Price_1_1.52', 'Price_5_Other',
       'Price_3_1.96', 'Price_3_Other', 'Last_Inc_Brand_2', 'Price_4_2.24',
       'Price_2_1.89', 'Price_4_2.16', 'Price_4_2.21', 'Price_1_1.47', 
       'Price_5_2.67', 'Price_3_2.07', 'Price_5_2.8', 'Price_2_1.82', 
        'Price_3_2.06', 'Price_3_2.02', 'Price_2_1.52', 
       'Price_3_1.98', 'Price_2_1.83', 'Price_2_1.57', 
        'Price_4_2.12', 'Price_1_1.39', 'Price_4_2.09', 'Price_1_1.27',
       'Price_4_2.18', 'Price_5_2.68', 'Price_5_2.73',
       'Price_1_1.36', 'Price_3_1.94', 'Price_2_1.5', 'Price_4_1.97',
       'Price_4_2.26','Price_2_1.87', 'Price_4_1.98', 'Price_5_2.58', 
        'Price_5_2.78', 'Price_2_1.51',
       'Price_5_2.61', 'Price_2_1.85', 'Price_4_2.15', 'Price_5_2.59',
       'Price_2_1.86', 'Price_1_1.37', 'Price_2_1.35', 'Price_4_1.96',
       'Price_5_2.71', 'Price_5_2.62', 'Price_2_1.88', 'Price_2_1.8',
       'Price_5_2.76', 'Price_3_2.09', 'Price_3_2.01', 'Price_5_2.65',
       'Price_5_2.77', 'Price_5_2.49', 'Price_5_2.63',
       'Price_4_Other', 'Price_1_1.4', 'Price_2_1.26','Price_3_1.93', 
       'Price_1_1.41', 'Price_2_1.84', 'Price_2_1.58', 'Price_1_1.49',
       'Price_2_1.56', 'Price_1_1.26', 'Price_3_1.97', 'Price_1_1.51',
       'Price_5_2.69', 'Price_2_Other', 'Price_1_1.19', 'Price_5_2.79',
        'Price_1_1.46'] + excluding + other

X_final = X.drop(drop, axis=1)

In [88]:
m1 = Logit_model(X_final, y)

In [89]:
m1_results = m1.statLogit()
m1_results.summary2()

Optimization terminated successfully.
         Current function value: 0.083453
         Iterations 9


<class 'statsmodels.iolib.summary2.Summary'>
"""
                           Results: Logit
====================================================================
Model:                Logit            Pseudo R-squared: 0.102      
Dependent Variable:   Incidence_1      AIC:              5644.0645  
Date:                 2020-12-10 14:46 BIC:              5745.1576  
No. Observations:     33672            Log-Likelihood:   -2810.0    
Df Model:             11               LL-Null:          -3130.7    
Df Residuals:         33660            LLR p-value:      2.0941e-130
Converged:            1.0000           Scale:            1.0000     
No. Iterations:       9.0000                                        
--------------------------------------------------------------------
                     Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
--------------------------------------------------------------------
Price_3             -2.0446   0.0394 -51.9191 0.0000 -2.1218 -1.9674
Last_Inc_Brand_3     3.7410   0.1760  21.2550 0.0000  3.3961  4.0860
Last_Inc_Quantity_1 -0.7753   0.1545  -5.0179 0.0000 -1.0782 -0.4725
Promotion_3_1        0.5256   0.1626   3.2328 0.0012  0.2070  0.8443
Education_2          0.3592   0.1022   3.5132 0.0004  0.1588  0.5596
Settlement size_2   -0.4160   0.0996  -4.1779 0.0000 -0.6112 -0.2209
Price_3_1.99        -0.2607   0.1345  -1.9381 0.0526 -0.5243  0.0029
change_in_P1         1.7568   0.6173   2.8457 0.0044  0.5468  2.9667
change_in_P2         0.6780   0.3075   2.2047 0.0275  0.0753  1.2807
change_in_P5         3.0537   1.0776   2.8339 0.0046  0.9417  5.1657
odd_price_above_5    0.0397   0.1048   0.3790 0.7047 -0.1657  0.2452
even_price_above_5   0.1224   0.1184   1.0339 0.3012 -0.1097  0.3545
====================================================================

"""

In [90]:
m1.vif().head()

,variables,VIF
0,Price_3,3.60
10,odd_price_above_5,2.93
6,Price_3_1.99,1.80
5,Settlement size_2,1.46
11,even_price_above_5,1.46


In [91]:
m1_results.pvalues.sort_values(ascending=False).keys()

Index(['odd_price_above_5', 'even_price_above_5', 'Price_3_1.99',
       'change_in_P2', 'change_in_P5', 'change_in_P1', 'Promotion_3_1',
       'Education_2', 'Settlement size_2', 'Last_Inc_Quantity_1',
       'Last_Inc_Brand_3', 'Price_3'],
      dtype='object')

### Model 2 all brands included

In [92]:
dataset_allBrands = dataset.astype('float')
target = 'Incidence_1'

In [93]:
odd_price = []
even_price = []
price_ends_with_9 = []

for column in X[price_dummies.columns].columns:
    try:
        is_odd = int(column[-1]) % 2
        is_9 = column[-1] == '9'
        is_aeq_5 = int(column[-1]) >= 5
        if is_odd and is_aeq_5:
            odd_price.append(column)
        elif (not is_odd) and is_aeq_5:
            even_price.append(column)
        if is_9:
            price_ends_with_9.append(column)
    except:
        pass

dataset_allBrands['ends_with_9'] = dataset_allBrands[price_ends_with_9].sum(axis=1).apply(lambda x: int(x!=0)) 
dataset_allBrands['odd_price_above_5'] = dataset_allBrands[odd_price].sum(axis=1).apply(lambda x: int(x!=0))      
dataset_allBrands['even_price_above_5'] = dataset_allBrands[even_price].sum(axis=1).apply(lambda x: int(x!=0))

In [94]:
y = dataset_allBrands[target]
X = dataset_allBrands.drop(target, axis=1)
X = sm.add_constant(X)

In [95]:
# Dropping columns that cause multicollinearity
excluding = ['ID', 'Quantity','Brand_1', 'Brand_2', 'Brand_4', 'Brand_5', 'Brand_3', 
            'const', 'Price_4', 'change_in_P5', 'change_in_P3', 'Price_2', 
            'Price_3', 'Price_5', 'Age', 'Income', 'Promotion_2_1', 
             'Last_Inc_Brand_3', 'change_in_P4','Promotion_3_1',
            'Occupation_1', 'Education_1', 'Marital status_1', 'Promotion_1_1',
            'Day', 'change_in_P1', 'ends_with_9', 'Settlement size_1', 
             'Sex_1', 'Last_Inc_Brand_1', 'odd_price_above_5']


other = ['Price_1_1.21', 'Price_1_1.33', 'Price_1_1.34', 'Price_1_1.35',
       'Price_1_1.42', 'Price_1_1.5', 'Price_1_Other', 'Price_2_1.9',
       'Price_5_2.64', 'Price_5_2.66', 'Price_5_2.7', 'Price_3_2.0', 
        'Price_3_1.95', 'Price_3_1.91', 'Price_3_1.91', 'Price_1_1.48']

drop = ['Price_2_1.81', 'Price_4_1.9', 'Price_1_1.52', 'Price_5_Other',
       'Price_3_1.96', 'Price_3_Other', 'Last_Inc_Brand_2', 'Price_4_2.24',
       'Price_2_1.89', 'Price_4_2.16', 'Price_4_2.21', 'Price_1_1.47', 
       'Price_5_2.67', 'Price_3_2.07', 'Price_5_2.8', 'Price_2_1.82', 
        'Price_3_2.06', 'Price_3_2.02', 'Price_2_1.52', 
       'Price_3_1.98', 'Price_2_1.83', 'Price_2_1.57', 
        'Price_4_2.12', 'Price_1_1.39', 'Price_4_2.09', 'Price_1_1.27',
       'Price_4_2.18', 'Price_5_2.68', 'Price_5_2.73',
       'Price_1_1.36', 'Price_3_1.94', 'Price_2_1.5', 'Price_4_1.97',
       'Price_4_2.26','Price_2_1.87', 'Price_4_1.98', 'Price_5_2.58', 
        'Price_5_2.78', 'Price_2_1.51',
       'Price_5_2.61', 'Price_2_1.85', 'Price_4_2.15', 'Price_5_2.59',
       'Price_2_1.86', 'Price_1_1.37', 'Price_2_1.35', 'Price_4_1.96',
       'Price_5_2.71', 'Price_5_2.62', 'Price_2_1.88', 'Price_2_1.8',
       'Price_5_2.76', 'Price_3_2.09', 'Price_3_2.01', 'Price_5_2.65',
       'Price_5_2.77', 'Price_5_2.49', 'Price_5_2.63',
       'Price_4_Other', 'Price_1_1.4', 'Price_2_1.26','Price_3_1.93', 
       'Price_1_1.41', 'Price_2_1.84', 'Price_2_1.58', 'Price_1_1.49',
       'Price_2_1.56', 'Price_1_1.26', 'Price_3_1.97', 'Price_1_1.51',
       'Price_5_2.69', 'Price_2_Other', 'Price_1_1.19', 'Price_5_2.79',
        'Price_1_1.46'] + excluding + other

X_final = X.drop(drop, axis=1)

In [96]:
m2 = Logit_model(X_final, y)
m2_results = m2.statLogit()
m2_results.summary2()

Optimization terminated successfully.
         Current function value: 0.527487
         Iterations 5


<class 'statsmodels.iolib.summary2.Summary'>
"""
                           Results: Logit
====================================================================
Model:               Logit             Pseudo R-squared:  0.062     
Dependent Variable:  Incidence_1       AIC:               46464.9292
Date:                2020-12-10 14:46  BIC:               46577.9301
No. Observations:    44019             Log-Likelihood:    -23219.   
Df Model:            12                LL-Null:           -24756.   
Df Residuals:        44006             LLR p-value:       0.0000    
Converged:           1.0000            Scale:             1.0000    
No. Iterations:      5.0000                                         
--------------------------------------------------------------------
                     Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
--------------------------------------------------------------------
Price_1             -1.0553   0.0182 -58.0351 0.0000 -1.0910 -1.0197
Last_Inc_Brand_4     0.4414   0.0541   8.1661 0.0000  0.3355  0.5473
Last_Inc_Brand_5     1.0544   0.0464  22.7421 0.0000  0.9635  1.1453
Last_Inc_Quantity_1  0.5357   0.0345  15.5357 0.0000  0.4681  0.6032
Promotion_4_1        0.1312   0.0361   3.6358 0.0003  0.0605  0.2020
Promotion_5_1        0.3956   0.0584   6.7751 0.0000  0.2811  0.5100
Education_2         -0.1638   0.0346  -4.7319 0.0000 -0.2316 -0.0959
Education_3          1.3435   0.1366   9.8337 0.0000  1.0757  1.6113
Occupation_2         0.3137   0.0303  10.3461 0.0000  0.2543  0.3731
Settlement size_2   -0.0777   0.0253  -3.0658 0.0022 -0.1273 -0.0280
Price_3_1.99        -0.1429   0.0302  -4.7398 0.0000 -0.2020 -0.0838
change_in_P2        -0.9085   0.0889 -10.2165 0.0000 -1.0828 -0.7342
even_price_above_5   0.0771   0.0250   3.0833 0.0020  0.0281  0.1261
====================================================================

"""

In [97]:
m2.vif().head()

,variables,VIF
0,Price_1,4.70
12,even_price_above_5,3.10
3,Last_Inc_Quantity_1,2.53
2,Last_Inc_Brand_5,1.80
9,Settlement size_2,1.53


In [98]:
m2_results.pvalues.sort_values(ascending=False).keys()

Index(['Settlement size_2', 'even_price_above_5', 'Promotion_4_1',
       'Education_2', 'Price_3_1.99', 'Promotion_5_1', 'Last_Inc_Brand_4',
       'Education_3', 'change_in_P2', 'Occupation_2', 'Last_Inc_Quantity_1',
       'Last_Inc_Brand_5', 'Price_1'],
      dtype='object')

### Performing PCA and K-means clustering

In [99]:
price_dummies.head()

,Price_1_1.19,Price_1_1.21,Price_1_1.26,Price_1_1.27,Price_1_1.33,Price_1_1.34,Price_1_1.35,Price_1_1.36,Price_1_1.37,Price_1_1.39,...,Price_5_2.69,Price_5_2.7,Price_5_2.71,Price_5_2.73,Price_5_2.76,Price_5_2.77,Price_5_2.78,Price_5_2.79,Price_5_2.8,Price_5_Other
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
pca = PCA(n_components=30)
principalComponents = pca.fit_transform(price_dummies)


scaler_c = StandardScaler()
principalComponents = scaler_c.fit_transform(principalComponents)

dummy_components = pd.DataFrame(principalComponents).add_prefix('C')
dummy_components.head()

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29
0,0.03,0.12,-0.19,-0.38,-1.34,1.73,-1.01,0.83,1.52,3.30,...,-1.05,0.18,-0.29,0.44,0.61,-0.30,0.52,0.52,0.15,-0.10
1,0.36,-0.73,0.90,-1.46,-1.47,1.88,-2.03,-0.26,0.10,0.26,...,-0.58,0.61,1.11,0.11,0.32,0.15,-1.29,0.37,0.85,-0.02
2,0.36,-0.73,0.90,-1.46,-1.47,1.88,-2.03,-0.26,0.10,0.26,...,-0.58,0.61,1.11,0.11,0.32,0.15,-1.29,0.37,0.85,-0.02
3,-0.20,-0.60,1.06,-0.90,-0.84,1.28,-1.52,0.12,0.08,1.70,...,-1.22,1.46,-0.10,0.74,0.48,0.30,-1.60,0.55,0.78,0.06
4,0.36,-0.78,0.89,-1.44,-1.48,1.87,-2.02,-0.24,0.09,0.26,...,-0.83,0.95,0.14,0.67,0.43,0.36,-1.28,0.23,0.85,0.19


In [101]:
pca.explained_variance_ratio_.sum()

0.8116573647786072

In [102]:
# Sum_of_squared_distances = []
# kmeans_data = price_dummies

# K = range(1,40)
# for k in K:
#     km = KMeans(n_clusters=k)
#     km = km.fit(kmeans_data)
#     Sum_of_squared_distances.append(km.inertia_)

# plt.plot(K, Sum_of_squared_distances, 'bx-')
# plt.xlabel('k')
# plt.ylabel('Sum_of_squared_distances')
# plt.title('Elbow Method For Optimal k')
# plt.show()

# # Conclusion: use 30

In [103]:
kmeans_clusters = KMeans(n_clusters=30, random_state=42).fit_transform(price_dummies)

scaler_k = StandardScaler()
kmeans_clusters = scaler_k.fit_transform(kmeans_clusters)

dummy_clusters = pd.DataFrame(kmeans_clusters).add_prefix('K')
dummy_clusters.head()

,K0,K1,K2,K3,K4,K5,K6,K7,K8,K9,...,K20,K21,K22,K23,K24,K25,K26,K27,K28,K29
0,0.26,0.76,0.73,0.41,0.51,-1.55,0.37,-0.67,0.71,0.85,...,0.44,0.87,0.34,0.49,-0.35,0.61,0.30,0.39,0.37,-1.05
1,0.48,-0.02,-0.16,-0.66,0.14,-1.72,-0.26,0.41,-0.22,-0.54,...,0.44,-0.06,0.34,-0.60,-0.41,-0.27,-0.13,0.39,0.27,-4.95
2,0.48,-0.02,-0.16,-0.66,0.14,-1.72,-0.26,0.41,-0.22,-0.54,...,0.44,-0.06,0.34,-0.60,-0.41,-0.27,-0.13,0.39,0.27,-4.95
3,0.54,-0.02,0.15,0.33,0.14,-0.83,0.37,0.41,-0.33,-0.31,...,0.44,0.01,0.34,0.49,-0.41,0.61,0.69,0.39,0.49,-3.46
4,0.54,-0.02,-0.16,-0.66,0.14,-1.76,0.37,0.41,-0.22,-0.54,...,0.44,-0.06,0.34,-0.60,-0.41,-0.27,-0.13,0.39,0.27,-4.87


In [104]:
dataset_with_components = pd.concat([dataset.drop(price_dummies.columns, axis=1), dummy_components], axis=1)
dataset_with_clusters = pd.concat([dataset.drop(price_dummies.columns, axis=1), dummy_clusters], axis=1)

In [105]:
# components_to_explain = ['C18', 'C1', 'C4', 'C21', 'C6', 'C8', 'C5']

# explain = explain_components(price_dummies, dummy_components[components_to_explain])
# explain.loc[1:].head(12)

## Saving the dataset

In [106]:
dataset.head()

,ID,Day,Quantity,Price_1,Price_2,Price_3,Price_4,Price_5,Age,Income,...,Price_5_2.77,Price_5_2.78,Price_5_2.79,Price_5_2.8,Price_5_Other,change_in_P1,change_in_P2,change_in_P3,change_in_P4,change_in_P5
0,100000001,1,0,1.59,1.87,2.01,2.09,2.66,67,124670,...,0,0,0,0,0,-0.05,0.01,-0.01,0.00,0.00
1,100000001,11,0,1.51,1.89,1.99,2.09,2.66,67,124670,...,0,0,0,0,0,-0.05,0.01,-0.01,0.00,0.00
2,100000001,12,0,1.51,1.89,1.99,2.09,2.66,67,124670,...,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00
3,100000001,16,0,1.52,1.89,1.98,2.09,2.66,67,124670,...,0,0,0,0,0,0.01,0.00,-0.01,0.00,0.00
4,100000001,18,0,1.52,1.89,1.99,2.09,2.66,67,124670,...,0,0,0,0,0,0.00,0.00,0.01,0.00,0.00


In [107]:
dataset.to_csv('data/charm_pricing.csv')

## Reporting results

In [108]:
# def ends_9(num):
#     return str(num)[-1] == '9'

# def odd_above_5(num):
#     last_num = int(str(num)[-1])
#     is_odd = bool(last_num % 2)
#     is_above_5 = last_num >= 5
#     return is_odd and is_above_5

# def even_above_5(num):
#     last_num = int(str(num)[-1])
#     is_odd = bool(last_num % 2)
#     is_above_5 = last_num >= 5
#     return (not is_odd) and is_above_5

# # 'ends_with_9', 'odd_price_above_5', 'even_price_above_5'
# price_cols = ['Price_1', 'Price_2', 'Price_3', 'Price_4', 'Price_5']
def model_vars(list_of_models):
    variables = []
    for model in list_of_models:
        variables.extend(model.params.index.to_list())
    return variables

variables = model_vars([m1_results, m2_results])
params_of_interest = ['Price_1', 'Price_3','Price_3_1.99', 'even_price_above_5',
                     'odd_price_above_5', 'Education_3', 'Occupation_2', 
                      'Last_Inc_Brand_5'] + [target]

pd.options.display.float_format = '{:,.2f}'.format

In [109]:
# 'Last_Inc_Brand', 'Education' 'Occupation', 'Settlement size'
# dummyC  = {'Brand','Sex', 'Last_Inc_Brand', 'Education', 'Occupation', 'Settlement size'}

SummaryStatistics = data.drop('ID', axis=1).describe().transpose()
print(SummaryStatistics.to_latex(label='SummaryStatistics',caption='Summary Statistics'))
# SummaryStatistics

\begin{table}
\centering
\caption{Summary Statistics}
\label{SummaryStatistics}
\begin{tabular}{lrrrrrrrr}
\toprule
{} &     count &       mean &       std &       min &        25\% &        50\% &        75\% &        max \\
\midrule
Day               & 58,693.00 &     349.43 &    212.05 &      1.00 &     161.00 &     343.00 &     530.00 &     730.00 \\
Incidence         & 58,693.00 &       0.25 &      0.43 &      0.00 &       0.00 &       0.00 &       0.00 &       1.00 \\
Brand             & 58,693.00 &       0.84 &      1.63 &      0.00 &       0.00 &       0.00 &       0.00 &       5.00 \\
Quantity          & 58,693.00 &       0.69 &      1.50 &      0.00 &       0.00 &       0.00 &       0.00 &      15.00 \\
Last\_Inc\_Brand    & 58,693.00 &       0.84 &      1.63 &      0.00 &       0.00 &       0.00 &       0.00 &       5.00 \\
Last\_Inc\_Quantity & 58,693.00 &       0.25 &      0.43 &      0.00 &       0.00 &       0.00 &       0.00 &       1.00 \\
Price\_1           & 58,693.0

In [110]:
# dataset_allBrands[[target]+variables].corr()
CorrelationMatrix = dataset_allBrands[params_of_interest].corr()
print(CorrelationMatrix.to_latex(label='CorrelationMatrix',caption='Correlation Matrix'))

\begin{table}
\centering
\caption{Correlation Matrix}
\label{CorrelationMatrix}
\begin{tabular}{lrrrrrrrrr}
\toprule
{} &  Price\_1 &  Price\_3 &  Price\_3\_1.99 &  even\_price\_above\_5 &  odd\_price\_above\_5 &  Education\_3 &  Occupation\_2 &  Last\_Inc\_Brand\_5 &  Incidence\_1 \\
\midrule
Price\_1            &     1.00 &    -0.00 &          0.04 &                0.08 &               0.09 &        -0.00 &         -0.00 &             -0.02 &        -0.05 \\
Price\_3            &    -0.00 &     1.00 &         -0.17 &                0.24 &              -0.13 &        -0.01 &         -0.00 &             -0.01 &        -0.00 \\
Price\_3\_1.99       &     0.04 &    -0.17 &          1.00 &                0.05 &               0.08 &        -0.00 &         -0.00 &             -0.01 &        -0.02 \\
even\_price\_above\_5 &     0.08 &     0.24 &          0.05 &                1.00 &              -0.12 &         0.00 &          0.00 &             -0.00 &        -0.00 \\
odd\_price\_above\_5  

In [111]:
m1_results.params = pd.Series(m1_results.get_margeff().margeff,index=m1_results.params.index)
m1_results.bse = pd.Series(m1_results.get_margeff().margeff_se,index=m1_results.bse.index)

m2_results.params = pd.Series(m2_results.get_margeff().margeff,index=m2_results.params.index)
m2_results.bse = pd.Series(m2_results.get_margeff().margeff_se,index=m2_results.bse.index)

# stargazer = Stargazer([m1_results,m2_results])
# pr2s = [m1_results.prsquared.round(4), m2_results.prsquared.round(4)]
# stargazer.title("Logit Marginal Effects")
# stargazer.custom_columns(labels = ['Model 1', 'Model 2'],separators=[1, 1])
# stargazer.add_line('Pseudo R2', pr2s, 'ft')
# stargazer.show_adj_r2 = False
# stargazer.show_r2 = False
# stargazer.show_model_nums = False
# stargazer.show_f_statistic = False
# stargazer.show_residual_std_err = False

In [169]:
info = {
    "N":lambda x:"{0:d}".format(int(x.nobs)),
    'Pseudo_R2': lambda x:"{:.2f}".format(x.prsquared),
    'Accuracy': lambda x: "{:.2%}".format(accuracy_score(x.model.endog,x.predict().round())),
    'Predicted 1s': lambda x: "{0:d}".format(int(x.predict().round().sum()))
}

view = summary_col([m1_results, m2_results], model_names=('Brand 3', 'All Brands'), 
           stars=True, info_dict=info, float_format='%.3f', regressor_order=params_of_interest)

view.add_title('Logit Marginal Effects')
view.extra_txt = ['* p<.1, ** p<.05, ***p<.01']

In [170]:
print(view.as_latex(label='LogitMF'))

\begin{table}
\caption{Logit Marginal Effects}
\label{LogitMF}
\begin{center}
\begin{tabular}{lll}
\hline
                       & Brand 3   & All Brands  \\
\hline
Price\_1               &           & -0.183***   \\
                       &           & (0.003)     \\
Price\_3               & -0.035*** &             \\
                       & (0.001)   &             \\
Price\_3\_1.99         & -0.005*   & -0.025***   \\
                       & (0.002)   & (0.005)     \\
even\_price\_above\_5  & 0.002     & 0.013***    \\
                       & (0.002)   & (0.004)     \\
odd\_price\_above\_5   & 0.001     &             \\
                       & (0.002)   &             \\
Education\_3           &           & 0.232***    \\
                       &           & (0.024)     \\
Occupation\_2          &           & 0.054***    \\
                       &           & (0.005)     \\
Last\_Inc\_Brand\_5    &           & 0.182***    \\
                       &           & (0.008)     \\
Las